In [1]:
import os
import torch
import torchvision
import torch.nn as nn
from torchvision import transforms
from torchvision.utils import save_image

In [2]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyper-parameters
latent_size = 64
hidden_size = 256
image_size = 784
num_epochs = 200
batch_size = 100
sample_dir = 'samples'

# Create a directory if not exists
if not os.path.exists(sample_dir):
    os.makedirs(sample_dir)

# Image processing
# transform = transforms.Compose([
#                 transforms.ToTensor(),
#                 transforms.Normalize(mean=(0.5, 0.5, 0.5),   # 3 for RGB channels
#                                      std=(0.5, 0.5, 0.5))])
transform = transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.5],   # 1 for greyscale channels
                                     std=[0.5])])

# MNIST dataset
mnist = torchvision.datasets.MNIST(root='../../data/',
                                   train=True,
                                   transform=transform,
                                   download=True)


0it [00:00, ?it/s]

Extracting ../../data/MNIST/raw/train-images-idx3-ubyte.gz to ../../data/MNIST/raw


0it [00:00, ?it/s]

Extracting ../../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../../data/MNIST/raw


0it [00:00, ?it/s]

Extracting ../../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../../data/MNIST/raw


0it [00:00, ?it/s]

Extracting ../../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../../data/MNIST/raw
Processing...
Done!


/pytorch/torch/csrc/utils/tensor_numpy.cpp:141: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program.


In [3]:
# Data loader
data_loader = torch.utils.data.DataLoader(dataset=mnist,
                                          batch_size=batch_size, 
                                          shuffle=True)

# Discriminator
D = nn.Sequential(
    nn.Linear(image_size, hidden_size),
    nn.LeakyReLU(0.2),
    nn.Linear(hidden_size, hidden_size),
    nn.LeakyReLU(0.2),
    nn.Linear(hidden_size, 1),
    nn.Sigmoid())

# Generator 
G = nn.Sequential(
    nn.Linear(latent_size, hidden_size),
    nn.ReLU(),
    nn.Linear(hidden_size, hidden_size),
    nn.ReLU(),
    nn.Linear(hidden_size, image_size),
    nn.Tanh())

# Device setting
D = D.to(device)
G = G.to(device)

# Binary cross entropy loss and optimizer
criterion = nn.BCELoss()
d_optimizer = torch.optim.Adam(D.parameters(), lr=0.0002)
g_optimizer = torch.optim.Adam(G.parameters(), lr=0.0002)

def denorm(x):
    out = (x + 1) / 2
    return out.clamp(0, 1)

def reset_grad():
    d_optimizer.zero_grad()
    g_optimizer.zero_grad()

In [4]:
# Start training
total_step = len(data_loader)
for epoch in range(num_epochs):
    for i, (images, _) in enumerate(data_loader):
        images = images.reshape(batch_size, -1).to(device)
        
        # Create the labels which are later used as input for the BCE loss
        real_labels = torch.ones(batch_size, 1).to(device)
        fake_labels = torch.zeros(batch_size, 1).to(device)

        # ================================================================== #
        #                      Train the discriminator                       #
        # ================================================================== #

        # Compute BCE_Loss using real images where BCE_Loss(x, y): - y * log(D(x)) - (1-y) * log(1 - D(x))
        # Second term of the loss is always zero since real_labels == 1
        outputs = D(images)
        d_loss_real = criterion(outputs, real_labels)
        real_score = outputs
        
        # Compute BCELoss using fake images
        # First term of the loss is always zero since fake_labels == 0
        z = torch.randn(batch_size, latent_size).to(device)
        fake_images = G(z)
        outputs = D(fake_images)
        d_loss_fake = criterion(outputs, fake_labels)
        fake_score = outputs
        
        # Backprop and optimize
        d_loss = d_loss_real + d_loss_fake
        reset_grad()
        d_loss.backward()
        d_optimizer.step()
        
        # ================================================================== #
        #                        Train the generator                         #
        # ================================================================== #

        # Compute loss with fake images
        z = torch.randn(batch_size, latent_size).to(device)
        fake_images = G(z)
        outputs = D(fake_images)
        
        # We train G to maximize log(D(G(z)) instead of minimizing log(1-D(G(z)))
        # For the reason, see the last paragraph of section 3. https://arxiv.org/pdf/1406.2661.pdf
        g_loss = criterion(outputs, real_labels)
        
        # Backprop and optimize
        reset_grad()
        g_loss.backward()
        g_optimizer.step()
        
        if (i+1) % 200 == 0:
            print('Epoch [{}/{}], Step [{}/{}], d_loss: {:.4f}, g_loss: {:.4f}, D(x): {:.2f}, D(G(z)): {:.2f}' 
                  .format(epoch, num_epochs, i+1, total_step, d_loss.item(), g_loss.item(), 
                          real_score.mean().item(), fake_score.mean().item()))
    
    # Save real images
    if (epoch+1) == 1:
        images = images.reshape(images.size(0), 1, 28, 28)
        save_image(denorm(images), os.path.join(sample_dir, 'real_images.png'))
    
    # Save sampled images
    fake_images = fake_images.reshape(fake_images.size(0), 1, 28, 28)
    save_image(denorm(fake_images), os.path.join(sample_dir, 'fake_images-{}.png'.format(epoch+1)))

# Save the model checkpoints 
torch.save(G.state_dict(), 'G.ckpt')
torch.save(D.state_dict(), 'D.ckpt')

Epoch [0/200], Step [200/600], d_loss: 0.0776, g_loss: 3.8968, D(x): 0.99, D(G(z)): 0.06
Epoch [0/200], Step [400/600], d_loss: 0.0432, g_loss: 5.7590, D(x): 0.99, D(G(z)): 0.03
Epoch [0/200], Step [600/600], d_loss: 0.0428, g_loss: 4.6953, D(x): 0.99, D(G(z)): 0.04
Epoch [1/200], Step [200/600], d_loss: 0.0250, g_loss: 5.7485, D(x): 0.99, D(G(z)): 0.01
Epoch [1/200], Step [400/600], d_loss: 0.5848, g_loss: 2.3060, D(x): 0.78, D(G(z)): 0.08
Epoch [1/200], Step [600/600], d_loss: 0.3046, g_loss: 4.1880, D(x): 0.93, D(G(z)): 0.14
Epoch [2/200], Step [200/600], d_loss: 0.0848, g_loss: 4.2656, D(x): 0.97, D(G(z)): 0.04
Epoch [2/200], Step [400/600], d_loss: 0.6015, g_loss: 3.3656, D(x): 0.85, D(G(z)): 0.22
Epoch [2/200], Step [600/600], d_loss: 0.4160, g_loss: 2.9631, D(x): 0.89, D(G(z)): 0.19
Epoch [3/200], Step [200/600], d_loss: 0.4831, g_loss: 3.1735, D(x): 0.80, D(G(z)): 0.08
Epoch [3/200], Step [400/600], d_loss: 0.3692, g_loss: 3.0865, D(x): 0.90, D(G(z)): 0.15
Epoch [3/200], Step [

Epoch [30/200], Step [600/600], d_loss: 0.4468, g_loss: 3.0392, D(x): 0.86, D(G(z)): 0.09
Epoch [31/200], Step [200/600], d_loss: 0.2448, g_loss: 4.0620, D(x): 0.93, D(G(z)): 0.12
Epoch [31/200], Step [400/600], d_loss: 0.2347, g_loss: 3.7623, D(x): 0.93, D(G(z)): 0.10
Epoch [31/200], Step [600/600], d_loss: 0.4013, g_loss: 3.4549, D(x): 0.93, D(G(z)): 0.14
Epoch [32/200], Step [200/600], d_loss: 0.2628, g_loss: 3.8739, D(x): 0.91, D(G(z)): 0.08
Epoch [32/200], Step [400/600], d_loss: 0.1462, g_loss: 5.1762, D(x): 0.98, D(G(z)): 0.10
Epoch [32/200], Step [600/600], d_loss: 0.1621, g_loss: 5.2293, D(x): 0.97, D(G(z)): 0.09
Epoch [33/200], Step [200/600], d_loss: 0.1869, g_loss: 6.8447, D(x): 0.94, D(G(z)): 0.06
Epoch [33/200], Step [400/600], d_loss: 0.2217, g_loss: 4.0414, D(x): 0.93, D(G(z)): 0.09
Epoch [33/200], Step [600/600], d_loss: 0.3617, g_loss: 4.3598, D(x): 0.88, D(G(z)): 0.09
Epoch [34/200], Step [200/600], d_loss: 0.2050, g_loss: 3.1760, D(x): 0.96, D(G(z)): 0.12
Epoch [34/

Epoch [61/200], Step [400/600], d_loss: 0.6997, g_loss: 2.4836, D(x): 0.87, D(G(z)): 0.33
Epoch [61/200], Step [600/600], d_loss: 0.6004, g_loss: 2.5590, D(x): 0.73, D(G(z)): 0.10
Epoch [62/200], Step [200/600], d_loss: 0.7523, g_loss: 1.9289, D(x): 0.72, D(G(z)): 0.21
Epoch [62/200], Step [400/600], d_loss: 0.5574, g_loss: 2.6116, D(x): 0.86, D(G(z)): 0.21
Epoch [62/200], Step [600/600], d_loss: 0.5482, g_loss: 2.1404, D(x): 0.92, D(G(z)): 0.29
Epoch [63/200], Step [200/600], d_loss: 0.7313, g_loss: 2.4090, D(x): 0.84, D(G(z)): 0.32
Epoch [63/200], Step [400/600], d_loss: 0.5878, g_loss: 2.0532, D(x): 0.84, D(G(z)): 0.21
Epoch [63/200], Step [600/600], d_loss: 0.6853, g_loss: 1.9363, D(x): 0.78, D(G(z)): 0.22
Epoch [64/200], Step [200/600], d_loss: 0.8014, g_loss: 2.5240, D(x): 0.70, D(G(z)): 0.16
Epoch [64/200], Step [400/600], d_loss: 0.5762, g_loss: 2.4669, D(x): 0.89, D(G(z)): 0.26
Epoch [64/200], Step [600/600], d_loss: 0.5727, g_loss: 2.7978, D(x): 0.91, D(G(z)): 0.28
Epoch [65/

Epoch [92/200], Step [200/600], d_loss: 0.6613, g_loss: 1.8790, D(x): 0.77, D(G(z)): 0.24
Epoch [92/200], Step [400/600], d_loss: 0.8658, g_loss: 1.7227, D(x): 0.70, D(G(z)): 0.28
Epoch [92/200], Step [600/600], d_loss: 0.7507, g_loss: 1.4503, D(x): 0.79, D(G(z)): 0.31
Epoch [93/200], Step [200/600], d_loss: 0.8965, g_loss: 1.3735, D(x): 0.77, D(G(z)): 0.35
Epoch [93/200], Step [400/600], d_loss: 0.8918, g_loss: 1.4783, D(x): 0.77, D(G(z)): 0.37
Epoch [93/200], Step [600/600], d_loss: 0.6934, g_loss: 1.9616, D(x): 0.76, D(G(z)): 0.21
Epoch [94/200], Step [200/600], d_loss: 0.7322, g_loss: 2.1803, D(x): 0.75, D(G(z)): 0.22
Epoch [94/200], Step [400/600], d_loss: 0.6545, g_loss: 2.0919, D(x): 0.77, D(G(z)): 0.22
Epoch [94/200], Step [600/600], d_loss: 0.7220, g_loss: 1.5770, D(x): 0.84, D(G(z)): 0.34
Epoch [95/200], Step [200/600], d_loss: 0.7627, g_loss: 1.8742, D(x): 0.74, D(G(z)): 0.26
Epoch [95/200], Step [400/600], d_loss: 0.8461, g_loss: 1.7702, D(x): 0.78, D(G(z)): 0.33
Epoch [95/

Epoch [122/200], Step [400/600], d_loss: 0.8127, g_loss: 1.3515, D(x): 0.83, D(G(z)): 0.39
Epoch [122/200], Step [600/600], d_loss: 0.8548, g_loss: 1.2376, D(x): 0.72, D(G(z)): 0.29
Epoch [123/200], Step [200/600], d_loss: 0.9028, g_loss: 1.6635, D(x): 0.74, D(G(z)): 0.32
Epoch [123/200], Step [400/600], d_loss: 0.7731, g_loss: 1.6619, D(x): 0.75, D(G(z)): 0.28
Epoch [123/200], Step [600/600], d_loss: 0.6069, g_loss: 1.8541, D(x): 0.81, D(G(z)): 0.25
Epoch [124/200], Step [200/600], d_loss: 0.7217, g_loss: 1.8141, D(x): 0.72, D(G(z)): 0.21
Epoch [124/200], Step [400/600], d_loss: 0.7176, g_loss: 1.6707, D(x): 0.76, D(G(z)): 0.25
Epoch [124/200], Step [600/600], d_loss: 0.7245, g_loss: 1.7932, D(x): 0.70, D(G(z)): 0.20
Epoch [125/200], Step [200/600], d_loss: 0.7428, g_loss: 2.1465, D(x): 0.73, D(G(z)): 0.24
Epoch [125/200], Step [400/600], d_loss: 0.7523, g_loss: 1.8284, D(x): 0.71, D(G(z)): 0.20
Epoch [125/200], Step [600/600], d_loss: 0.9520, g_loss: 1.5736, D(x): 0.73, D(G(z)): 0.35

Epoch [152/200], Step [600/600], d_loss: 0.8172, g_loss: 1.7073, D(x): 0.73, D(G(z)): 0.28
Epoch [153/200], Step [200/600], d_loss: 0.9619, g_loss: 1.1114, D(x): 0.75, D(G(z)): 0.36
Epoch [153/200], Step [400/600], d_loss: 0.8550, g_loss: 1.7575, D(x): 0.75, D(G(z)): 0.34
Epoch [153/200], Step [600/600], d_loss: 0.8848, g_loss: 1.7452, D(x): 0.69, D(G(z)): 0.27
Epoch [154/200], Step [200/600], d_loss: 0.8556, g_loss: 1.4383, D(x): 0.71, D(G(z)): 0.27
Epoch [154/200], Step [400/600], d_loss: 1.0925, g_loss: 1.4161, D(x): 0.71, D(G(z)): 0.40
Epoch [154/200], Step [600/600], d_loss: 0.7270, g_loss: 1.6575, D(x): 0.75, D(G(z)): 0.27
Epoch [155/200], Step [200/600], d_loss: 0.7194, g_loss: 1.5328, D(x): 0.75, D(G(z)): 0.25
Epoch [155/200], Step [400/600], d_loss: 1.0394, g_loss: 1.5395, D(x): 0.65, D(G(z)): 0.29
Epoch [155/200], Step [600/600], d_loss: 1.0302, g_loss: 1.5455, D(x): 0.69, D(G(z)): 0.33
Epoch [156/200], Step [200/600], d_loss: 0.9191, g_loss: 1.9605, D(x): 0.66, D(G(z)): 0.26

Epoch [183/200], Step [200/600], d_loss: 1.2103, g_loss: 1.6183, D(x): 0.61, D(G(z)): 0.33
Epoch [183/200], Step [400/600], d_loss: 0.8764, g_loss: 1.5232, D(x): 0.69, D(G(z)): 0.29
Epoch [183/200], Step [600/600], d_loss: 0.9229, g_loss: 1.5163, D(x): 0.67, D(G(z)): 0.28
Epoch [184/200], Step [200/600], d_loss: 0.9868, g_loss: 1.5363, D(x): 0.66, D(G(z)): 0.28
Epoch [184/200], Step [400/600], d_loss: 0.8357, g_loss: 1.5700, D(x): 0.77, D(G(z)): 0.34
Epoch [184/200], Step [600/600], d_loss: 0.9995, g_loss: 1.1856, D(x): 0.77, D(G(z)): 0.41
Epoch [185/200], Step [200/600], d_loss: 0.7739, g_loss: 1.8629, D(x): 0.69, D(G(z)): 0.23
Epoch [185/200], Step [400/600], d_loss: 1.0714, g_loss: 1.4397, D(x): 0.70, D(G(z)): 0.37
Epoch [185/200], Step [600/600], d_loss: 1.0246, g_loss: 1.9625, D(x): 0.69, D(G(z)): 0.32
Epoch [186/200], Step [200/600], d_loss: 1.0135, g_loss: 1.5630, D(x): 0.61, D(G(z)): 0.26
Epoch [186/200], Step [400/600], d_loss: 1.0960, g_loss: 1.3229, D(x): 0.68, D(G(z)): 0.36